### Deliverables 2 and 3

In [27]:
import pandas as pd
from sodapy import Socrata
import collections

pd.set_option('display.width', 1505)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

# to read from local static copy
#df = pd.read_csv("~/Downloads/Food_Inspections.csv")
# to pull most recent version and read from the SODA API
client = Socrata("data.cityofchicago.org", None)
results = client.get("4ijn-s7e5", limit=20000)
results_df = pd.DataFrame.from_records(results)


#Drop all rows that arent restauratnts, and those for which the facility type or the zip code are NaN
results_df = results_df.dropna(subset=['facility_type', 'zip'])
results_df['facility_type'] = results_df['facility_type'].str.lower()
results_df['results'] = results_df['results'].str.lower()
restaurants_only_bool = results_df['facility_type'].str.contains("rest")



#Drop all rows where the license is not > 0
valid_license_bool = (results_df['license_'] != 0)
failed_inspection_bool = (results_df['results'] == 'fail')
passed_inspection_bool = (results_df['results'] == 'pass')
restaurants_only_df = results_df[restaurants_only_bool & valid_license_bool & (failed_inspection_bool | passed_inspection_bool)]

#Drop the columns that we dont really care about
restaurants_only_df = restaurants_only_df.drop(columns=['inspection_id', 'aka_name', 'risk',
 'city', 'state', 'inspection_date', 'location', 
 ':@computed_region_awaf_s7ux', ':@computed_region_6mkv_f3dw', ':@computed_region_vrxf_vc4k',
 ':@computed_region_bdys_3d7i' ,':@computed_region_43wa_7qmu'])

print(restaurants_only_df.head())


                    dba_name license_ facility_type               address    zip        inspection_type results  \
1   TOP-NOTCH BEEFBURGER INC    50905    restaurant  2114-2116 W 95TH ST   60643                Canvass    fail   
3                     SUBWAY  1766064    restaurant    5853 S KEDZIE AVE   60629                Canvass    pass   
7                     SBARRO  2882176    restaurant     500 W MADISON ST   60661                License    pass   
8                 YAO'S CAFE  2835735    restaurant        601 W 43RD ST   60609  Canvass Re-Inspection    pass   
13    HALE FAMILY MCDONALD'S  2771132    restaurant       1951 E 95th ST   60617              Complaint    fail   

                                                                                                                                                                                                                                                                                                                     

### Deliverable 4

In [10]:
#create a new df that contains just the little india restaurants. Then get the counts of the passes and fails
little_india_df = restaurants_only_df.loc[restaurants_only_df['zip']=='60659']
little_india_failed = little_india_df['results'].value_counts()
#find the percentage of the fails out of the totals
little_india_fail_perc = little_india_failed['fail'] / little_india_failed.sum() * 100
print(little_india_fail_perc)

#Do the same as above but for Chinatown now. 
chinatown_df = restaurants_only_df.loc[restaurants_only_df['zip'] == '60608']
chinatown_failed = chinatown_df['results'].value_counts()
#get the percentage of the fails out of the totals. 
chinatown_fail_perc = chinatown_failed['fail'] / chinatown_failed.sum() * 100
print(chinatown_fail_perc)

#Chinatown has ~4% more fails than little india

22.413793103448278
39.130434782608695


### Deliverable 5
What is the main reason for all the restaurant inspections on Devon

In [13]:
#Create a var that checks if the restaurant address is on Devon ave
on_devon_bool = restaurants_only_df['address'].str.contains("DEVON")
#Use that bool to make a new dataframe
rests_on_devon = restaurants_only_df[on_devon_bool]
#print out the value counts for the reason of the ispection. The most common is Canvass, followed by Complaint and Re-Inspection
print(rests_on_devon['inspection_type'].value_counts())

Canvass                    38
Complaint                  21
Canvass Re-Inspection      14
License                    10
Short Form Complaint        9
Complaint Re-Inspection     7
License Re-Inspection       4
Name: inspection_type, dtype: int64


### Deliverable 6
What is the main reason for all the restautant inspecitons on Cermak

In [14]:
#Create a var that checks if the restaurant address is on Cermak
on_cermak_bool = restaurants_only_df['address'].str.contains("CERMAK")
#Use that bool to make a new dataframe
rests_on_cermak = restaurants_only_df[on_cermak_bool]
#print out the value counts for the reason of the ispection. The most common is Canvass, followed by Complaint and Re-Inspection
print(rests_on_cermak['inspection_type'].value_counts())

Canvass                    38
Complaint                  18
Complaint Re-Inspection    10
Canvass Re-Inspection       8
License                     7
Short Form Complaint        2
License Re-Inspection       1
Name: inspection_type, dtype: int64


### Deliverable 7
What is the main reason for the failed restaurant inspections on Devon?

In [35]:
# Get a var to check if the restaurant on devon failed and then make a new df with just the failed restaurants on Devon
failed_on_devon_bool = rests_on_devon['results'] == 'fail'
failed_on_devon = rests_on_devon[failed_on_devon_bool]
#Create a series with each restaurants violations, as a list, split at the | characters
vio_list = failed_on_devon['violations'].str.split('|')
#create a list to store the reasons
reasons = []
#iterate over the failure reasons and append the reason to the reasons list
for entry in vio_list:
    reasons.append(entry[0][:2])

#check the frequency of each reasona and store it as a dictionary
reason_freq = collections.Counter(reasons)
print(dict(sorted(reason_freq.items(), key=lambda item: item[1])))

{'28': 1, '9.': 1, '8.': 1, '16': 1, '2.': 2, '37': 2, '38': 3, '3.': 4, '1.': 6, '10': 6}


### Deliverable 8
What is the main reason for the failed restaurant inspections on Cermak

In [34]:
# Get a var to check if the restaurant on devon failed and then make a new df with just the failed restaurants on Cermak
failed_on_cermak_bool = rests_on_cermak['results'] == 'fail'
failed_on_cermak = rests_on_cermak[failed_on_cermak_bool]
#Create a series with each restaurants violations, as a list, split at the | characters
vio_list = failed_on_cermak['violations'].str.split('|')
#create a list to store the reasons
reasons = []
#iterate over the failure reasons and append the reason to the reasons list
for entry in vio_list:
    reasons.append(entry[0][:2])

#check the frequency of each reasona and store it as a dictionary
reason_freq = collections.Counter(reasons)
#Sort the frequency from least common to most
print(dict(sorted(reason_freq.items(), key=lambda item: item[1])))

{'59': 1, '16': 1, '29': 1, '22': 1, '41': 1, '5.': 1, '25': 1, '2.': 2, '3.': 2, '10': 2, '37': 3, '1.': 3, '38': 4}


### Deliverable 9
What is the most frequently failed restaurant on Devon

### Deliverable 10
What is the most frequently failed restaurank on Cermak

### Deliverable 11
What is the most frequently passed restaurant on Devon

### Deliverable 12
What is the most frequently passed restaurant on Cermak